<a href="https://colab.research.google.com/github/rocketwolf98/ColabCompilation/blob/main/parquet_6h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fireducks xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.


In [1]:
import fireducks.pandas as pd
import numpy as np
from google.colab import drive
import gc
import dask.dataframe as dd

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
path = '/content/drive/MyDrive/Compstat/part.0.parquet'

In [3]:
ddf = dd.read_parquet(path, engine='pyarrow')

In [4]:
del ddf['valid_time']

In [8]:
gc.collect()

25

In [7]:
ddf.head(5)

,time,t2m_C,d2m_C,rel_humidity,sp
0,2022-01-01,20.881500,16.519928,76.127480,102353.0625
1,2022-01-01,20.914703,16.539459,76.066689,102340.0625
2,2022-01-01,21.016266,16.689850,76.319595,102362.0625
3,2022-01-01,21.086578,16.873444,76.881660,102346.0625
4,2022-01-01,21.274078,16.949615,76.370537,102330.0625


In [6]:
ddf['month']=ddf['time'].dt.month
ddf['day']=ddf['time'].dt.day
ddf['year']=ddf['time'].dt.year
ddf['hh:mm']=ddf['time'].dt.time

In [9]:
ddf.head(5)

,time,t2m_C,d2m_C,rel_humidity,sp,month,day,year,hh:mm
0,2022-01-01,20.881500,16.519928,76.127480,102353.0625,1,1,2022,00:00:00
1,2022-01-01,20.914703,16.539459,76.066689,102340.0625,1,1,2022,00:00:00
2,2022-01-01,21.016266,16.689850,76.319595,102362.0625,1,1,2022,00:00:00
3,2022-01-01,21.086578,16.873444,76.881660,102346.0625,1,1,2022,00:00:00
4,2022-01-01,21.274078,16.949615,76.370537,102330.0625,1,1,2022,00:00:00


In [7]:
ddf["hh:mm"] = dd.to_datetime(ddf["hh:mm"], format="%H:%M:%S")
ddf_filtered=ddf[ddf["hh:mm"].dt.hour.isin([0,6,12,18])]

In [17]:
ddf_filtered.head()

,time,t2m_C,d2m_C,rel_humidity,sp,month,day,year,hh:mm
0,2022-01-01,20.881500,16.519928,76.127480,102353.0625,1,1,2022,1900-01-01
1,2022-01-01,20.914703,16.539459,76.066689,102340.0625,1,1,2022,1900-01-01
2,2022-01-01,21.016266,16.689850,76.319595,102362.0625,1,1,2022,1900-01-01
3,2022-01-01,21.086578,16.873444,76.881660,102346.0625,1,1,2022,1900-01-01
4,2022-01-01,21.274078,16.949615,76.370537,102330.0625,1,1,2022,1900-01-01


In [16]:
ddf_filtered.shape

(Delayed('int-e8cfb3f9-ff52-435d-a2d6-49425b46ac9c'), 9)

In [18]:
rows = ddf_filtered.shape[0].compute()  # Compute only the number of rows
cols = len(ddf_filtered.columns)  # Number of columns (doesn’t need compute)
print(f"Shape: ({rows}, {cols})")

Shape: (12823200, 9)


In [9]:
ddf = ddf_filtered.repartition(npartitions=1)
ddf.to_parquet('6-hours.parquet',engine='pyarrow', write_index='False')